In [1]:
import model
import os
from option import args
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import data
import utils
import numpy as np
import json
import h5py
loaded = {}
loaded_gt ={}

path,tot = "",""


def load_path(base_dir,ids):
    base_dir_7t = [base_dir + "/HCP_7T/" + i   for i in ids]
    base_dir_3t = [base_dir + "/HCP_3T/" + i   for i in ids]
    path_7t = {}
    path_3t = {}
    for i in base_dir_7t:
        path_7t[i[-6:]] = {"h5" : i + "/" + i[-6:] + ".h5"
                        , "GT" : i + "/" + i[-6:] + "_GT.h5"}
    for i in base_dir_3t:
        path_3t[i[-6:]] = {"h5" : i + "/" + i[-6:] + ".h5"
                        , "upsampled" : i + "/" + i[-6:] + "_upsampled.h5"
                        , "GT" : i + "/" + i[-6:] + "_GT.h5"}
    path = {'3T': path_3t, "7T":  path_7t}
    p = list(path_7t.keys())
    q = list(path_3t.keys())
    common = list(set(p) & set(q))

    return path,len(common)

def load_data(base_dir,ids):
    ids.sort()
    path,tot = load_path(base_dir,ids)
    act_ids = []
    for i in ids:
        name = path['3T'][i]['h5']
        if(not os.path.isfile(name)):
            continue
        res_vol = h5py.File(name, 'r')
        
        # print(res_vol.keys())
        loaded[i] = {'vol0':res_vol.get('volumes0')[:]
                            ,'mask':res_vol.get('mask')[:] }
        
        name = path['7T'][i]['GT']
        
        if(not os.path.isfile(name)):
            continue
        res = h5py.File(name, 'r')
        # print(res.keys())
        loaded_gt[i] = {'ADC':res.get('ADC')[:]
                            ,'FA':res.get('FA')[:] 
                            ,'color_FA':res.get('color_FA')[:] }
        
        
        res_vol.close()
        res.close()

        name = path['7T'][i]['h5']
        
        if(not os.path.isfile(name)):
            continue
        res = h5py.File(name, 'r')
        loaded_gt[i]['vol0'] = res.get('volumes0')[:]
        loaded_gt[i]['mask'] = res.get('mask')[:]
        res_vol.close()
        res.close()
        act_ids.append(i)
    return act_ids

number of common Subjects  171


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dir = "working_models"

In [3]:
paths = {}
max_ep = 0
for i in os.listdir(load_dir):
    if "check" not in i:
        temp = []
#         print(os.listdir(load_dir + "/" + i))
        for j in os.listdir(load_dir + "/" + i + "/model"):
            temp.append(load_dir + "/" + i + "/model/" + j)
            
            
            _,ep,hfen,psnr = j.split("_")
            ep = int(ep)
            if(max_ep < ep):
                latest_ep_path = load_dir + "/" + i + "/model/" + j
                max_ep = ep
                
            conf = load_dir + "/" + i + "/config.txt"
        paths[i] = (temp,conf,latest_ep_path) 

In [4]:
list_models = list(paths.keys())

In [5]:
paths

{'dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8': (['working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_25_0.5027_19.1527.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_3_0.6010_17.7611.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_6_0.5281_18.4563.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_27_0.4955_19.2090.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_7_0.5369_18.4709.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_4_0.6004_17.9686.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_13_0.5259_18.9297.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_58_0.5062_19.3343.pt',
   'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_9_0.4964

In [6]:
paths[list_models[0]][2]

'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_76_0.5122_19.3765.pt'

In [7]:
paths[list_models[0]][1]

'working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/config.txt'

In [8]:
list_models

['dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8',
 'dmri_rdn,False,conv,growth32,loss_0.5*MSE+0.5*L1,batch_size_8',
 'dmri_rdn,False,conv,growth64,loss_1*L1,batch_size_8',
 'dmri_rdn,False,conv,growth32,loss_1*L1,batch_size_8,variable_blk_size',
 'dmri_rdn,False,conv,growth32,loss_1*MSE,batch_size_8,variable_blk_size']

In [9]:
load_model = list_models[0]

In [10]:
load_model

'dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8'

In [11]:
with open(paths[load_model][1], 'r') as f:
    args.__dict__ = json.load(f)

In [12]:
args

Namespace(block_size=[32, 32, 4], var_blk_size=False, start_var=True, epochs=100, dir='/storage', batch_size=8, sort=True, debug=False, preload=True, ret_points=False, enable_thres=True, thres=0.6, rel_coord=False, patience=3, no_vols=10, test_vols=10, lr=0.005, max_lr=0.01, lr_decay=20, decay_type='step', gamma=0.8, optimizer='ADAM', momentum=0.9, beta1=0.9, beta2=0.999, epsilon=1e-08, weight_decay=0, start_epoch=0, loss='1*L1', skip_threshold=1000000.0, run_name='..', save='DTIArb', load='.', save_models=False, resume=0, print_every=20, save_every=30, cpu=False, gpu=0, seed=1, reset=False, pin_mem=False, model='dmri_rdn', in_chans=7, encoder='rdb', tv=False, attention=False, drop_prob=0, growth=128, model_type='3d', out_chans=5, RDNconfig='C', precision='single', cuda=True, scale=[1, 1, 1], offset=3, stable_epoch=1, tv_en=False)

In [13]:
# import json
# with open(paths[list_models[0]][1], 'w') as f:
#     json.dump(args.__dict__, f, indent=2)

In [14]:
curr_model = model.Model(args)
curr_model.load(paths[load_model][2])

Making model... here
load from model_working_models/dmri_rdn,False,conv,growth128,loss_1*L1,batch_size_8/model/model_76_0.5122_19.3765.pt.pt


In [15]:
ids = utils.get_ids()
ids.sort()
offset = 50
total_vols = args.no_vols+args.test_vols
temp = ids[:args.no_vols]
temp.extend(ids[offset:args.test_vols+offset])
ids = temp
print(ids)

['100610', '102311', '102816', '104416', '105923', '108323', '109123', '111312', '111514', '114823', '172130', '173334', '175237', '176542', '177140', '177645', '177746', '178142', '178243', '178647']


### Dataloader

In [16]:
loader = data.Data(args,ids= ids)

Loading Done


In [17]:
len(loader.training_dataset)

73

In [18]:
len(next(iter(loader.training_data)))

4

In [ ]:
temp[0].squeeze().shape[1:-1],temp[0].squeeze().shape

In [ ]:
np.where(np.asarray(temp[3].shape) == 8)[0][0]

In [ ]:
buf[1].shape,buf[0].shape,temp[3].shape

In [41]:
for batch, (lr,hr,scale,rel_coor) in enumerate(loader.training_data):

    lr_tensor = lr.squeeze().to('cuda').float()  # ranges from [0, 1]
    hr_tensor = hr.squeeze().to('cuda').float()  # ranges from [0, 1]
#     hr = hr.squeeze()
    rel_coor = rel_coor.squeeze().to('cuda').float()
    scale = np.asarray(scale[0,:])

    # print(rel_coor.shape)
    # tv_tensor = tv.squeeze().to('cuda').float()  # ranges from [0, 1]

    # print(lr_tensor.shape,hr_tensor.shape,scale)

    if(len(lr_tensor.shape) == 5):
        lr = torch.permute(lr_tensor, (0,4,1,2,3))
    else:
        lr = torch.permute(lr_tensor, (0,3,1,2))


    # inference
    pred = curr_model.forward(lr,scale,rel_coor)
    
    if(len(lr_tensor.shape) == 5):
        # print(pred.shape,lr.shape)
        pred_tensor = torch.permute(pred, (0,2,3,4,1)).float()
        # pred_tv_tensor = torch.permute(pred_tv, (0,2,3,4,1)).float()
    else:
        pred_tensor = torch.permute(pred, (0,2,3,1)).float()
        # pred_tv_tensor = torch.permute(pred_tv, (0,2,3,1)).float()
            
            
    break

In [121]:
pred_tensor.shape,hr_tensor.shape,lr_tensor.shape

(torch.Size([8, 32, 8, 32, 5]),
 torch.Size([8, 32, 8, 32, 5]),
 torch.Size([8, 32, 8, 32, 7]))

In [110]:

pred_temp = pred_tensor.reshape(8,temp)
hr_temp = hr_tensor.reshape(8,temp)

In [112]:
S_srsr

tensor([[1.0000, 0.9852, 0.9794, 0.9633, 0.9824, 0.9640, 0.9836, 0.9681],
        [0.9852, 1.0000, 0.9849, 0.9550, 0.9816, 0.9577, 0.9894, 0.9620],
        [0.9794, 0.9849, 1.0000, 0.9434, 0.9795, 0.9504, 0.9822, 0.9501],
        [0.9633, 0.9550, 0.9434, 1.0000, 0.9635, 0.9829, 0.9603, 0.9860],
        [0.9824, 0.9816, 0.9795, 0.9635, 1.0000, 0.9615, 0.9882, 0.9635],
        [0.9640, 0.9577, 0.9504, 0.9829, 0.9615, 1.0000, 0.9599, 0.9810],
        [0.9836, 0.9894, 0.9822, 0.9603, 0.9882, 0.9599, 1.0000, 0.9624],
        [0.9681, 0.9620, 0.9501, 0.9860, 0.9635, 0.9810, 0.9624, 1.0000]],
       grad_fn=<CopySlices>)

In [178]:
import torch
import torch.nn as nn
import numpy as np
import math
from torch import linalg as LA

class DCELoss(nn.Module):
    def __init__(self):
        super(DCELoss,self).__init__()


    def forward(self,pred,hr):
        S_srhr = torch.zeros((8,8))
        S_srsr = torch.zeros((8,8))
        M = torch.zeros((8,8))
        pred_temp = pred.reshape(8,temp)
        hr_temp = hr.reshape(8,temp)
        for i in range(8):
            for j in range(8):
                S_srhr[i][j] = torch.dot(pred_temp[i],hr_temp[j])/(LA.norm(pred_temp[i])* LA.norm(hr_temp[j]))
                S_srsr[i][j] = torch.dot(pred_temp[i],pred_temp[j])/(LA.norm(pred_temp[i])* LA.norm(pred_temp[j]))
                M[i][j] = min((-20)*torch.log10(LA.norm(hr_temp[i] - hr_temp[j])),0)
                
        Q_pos = torch.zeros((8))
        Q_neg = torch.zeros((8))
        n_thres = M.median()
        t_pos,t_neg = 0.5,0.5
        for i in range(8):
            for j in range(8):
                if( abs(M[i][j]) > abs(n_thres)):
                    Q_pos[i] +=  (( torch.exp(S_srsr[i][j]) + (2*torch.exp(S_srhr[i][j])) ) /t_pos )
                else:
                    Q_neg[i] += (( torch.exp(S_srsr[i][j]) + (2*torch.exp(S_srhr[i][j])) ) /t_neg )
                
        los = (-1/8) * torch.log(Q_pos/Q_neg).sum()
        
        return los

In [179]:
loss_fn = DCELoss()

In [180]:
loss_fn(pred_tensor,hr_tensor)

tensor(0.1657, grad_fn=<MulBackward0>)

In [177]:
float(metrics.peak_signal_noise_ratio(hr,pred,data_range=1))

ValueError: Input images must have the same dimensions.

In [126]:

for i in range(8):
    for j in range(8):

In [130]:
mean(M)

NameError: name 'mean' is not defined

In [133]:
M.mean()

tensor(-27.0891)

In [141]:
M.median(),M.mean()

(tensor(-30.9380), tensor(-27.0891))

In [143]:
M[M>M.median()]

tensor([  0.0000, -30.0437, -30.5460, -30.4371, -30.5017, -30.9214, -30.0437,
          0.0000, -30.7547, -30.5460, -30.7547,   0.0000,   0.0000, -29.1075,
        -30.7632, -28.8074, -30.4371,   0.0000, -30.4007, -29.1075,   0.0000,
        -29.2137, -30.5017, -30.7632, -30.4007,   0.0000, -30.3235, -30.9214,
        -28.8074, -29.2137, -30.3235,   0.0000])